## Csomagok betöltése

In [ ]:
import os
import json
import numpy as np
import torch
import networkx as nx
from tqdm import tqdm
from sklearn.metrics.pairwise import cosine_similarity
from scipy.spatial.distance import squareform
from transformers import SiglipProcessor, SiglipModel
import pandas as pd

# Paraméterek
included_classes = set(str(i) for i in range(1, 51))  # Csak 1–50 osztály
device = "cuda" if torch.cuda.is_available() else "cpu"

c:\Users\Adam\miniconda3\envs\main_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


## Kép embeddingek betöltése és szűrése

In [ ]:
# Képekhez tartozó embeddingek betöltése
embeddings = torch.load("../data/image_embeddings_siglip.pt")
infos = torch.load("../data/image_info_siglip.pt")

# Csak a kiválasztott osztályok
selected_indices = [i for i, info in enumerate(infos) if info["label"] in included_classes]
selected_embeddings = embeddings[selected_indices]
selected_labels = [infos[i]["label"] for i in selected_indices]

C:\Users\Adam\AppData\Local\Temp\ipykernel_7392\3304746802.py:2: FutureWarning: You are using `torch.load` with `weights_only=False` (the current default value), which uses the default pickle module implicitly. It is possible to construct malicious pickle data which will execute arbitrary code during unpickling (See https://github.com/pytorch/pytorch/blob/main/SECURITY.md#untrusted-models for more details). In a future release, the default value for `weights_only` will be flipped to `True`. This limits the functions that could be executed during unpickling. Arbitrary objects will no longer be allowed to be loaded via this mode unless they are explicitly allowlisted by the user via `torch.serialization.add_safe_globals`. We recommend you start setting `weights_only=True` for any use case where you don't have full control of the loaded file. Please open an issue on GitHub for any issues related to this experimental feature.
  embeddings = torch.load("C:/Users/Adam/Desktop/applied_ml/data

## Kép embeddingekből gráfépítés és perkolációs klaszterezés

In [3]:
epsilon = 0.06

# Cosine similarity → distance
similarity_matrix = cosine_similarity(selected_embeddings.numpy())
distance_matrix = 1 - similarity_matrix

# Gráf építése
G_img = nx.Graph()
for i in range(len(distance_matrix)):
    G_img.add_node(i, label=selected_labels[i])
    for j in range(i + 1, len(distance_matrix)):
        if distance_matrix[i][j] <= epsilon:
            G_img.add_edge(i, j)

# Komponensek
components_img = list(nx.connected_components(G_img))
print(f"Talált klaszterek száma (képek): {len(components_img)}")
for idx, comp in enumerate(components_img):
    comp_labels = {selected_labels[i] for i in comp}
    if len(comp_labels) > 1:
        print(f"⚠️ Klaszter {idx} több osztályt tartalmaz: {sorted(comp_labels)} (méret: {len(comp)})")
    else:
        print(f"Klaszter {idx}: tiszta ({list(comp_labels)[0]}) – {len(comp)} kép")

Talált klaszterek száma (képek): 1368
Klaszter 0: tiszta (1) – 1 kép
Klaszter 1: tiszta (1) – 6 kép
Klaszter 2: tiszta (1) – 1 kép
Klaszter 3: tiszta (1) – 1 kép
Klaszter 4: tiszta (1) – 2 kép
Klaszter 5: tiszta (1) – 1 kép
Klaszter 6: tiszta (1) – 1 kép
Klaszter 7: tiszta (1) – 1 kép
Klaszter 8: tiszta (1) – 1 kép
Klaszter 9: tiszta (1) – 2 kép
Klaszter 10: tiszta (1) – 1 kép
Klaszter 11: tiszta (1) – 1 kép
Klaszter 12: tiszta (1) – 1 kép
Klaszter 13: tiszta (1) – 1 kép
Klaszter 14: tiszta (1) – 1 kép
Klaszter 15: tiszta (1) – 1 kép
Klaszter 16: tiszta (1) – 1 kép
Klaszter 17: tiszta (1) – 1 kép
Klaszter 18: tiszta (1) – 1 kép
Klaszter 19: tiszta (1) – 1 kép
Klaszter 20: tiszta (2) – 1 kép
Klaszter 21: tiszta (2) – 1 kép
Klaszter 22: tiszta (2) – 3 kép
Klaszter 23: tiszta (2) – 1 kép
Klaszter 24: tiszta (2) – 1 kép
Klaszter 25: tiszta (2) – 1 kép
Klaszter 26: tiszta (2) – 1 kép
Klaszter 27: tiszta (2) – 1 kép
Klaszter 28: tiszta (2) – 2 kép
Klaszter 29: tiszta (2) – 1 kép
Klaszter 30:

## Egy klaszter részletesebb vizsgálata

In [5]:
from collections import Counter

target_idx = 340
cluster = components_img[target_idx]

# Label-ek gyűjtése
cluster_labels = [infos[selected_indices[i]]["label"] for i in cluster]
label_counts = Counter(cluster_labels)

print(f"\nEloszlás a {target_idx}. klaszterben:")
for label, count in sorted(label_counts.items(), key=lambda x: int(x[0])):
    print(f"- {count} db {label}-os kép")

# Részletes képinfók
print("\nKlaszter képei:")
for i in sorted(cluster):
    info = infos[selected_indices[i]]
    print(f"{i}: {info['path']} | label={info['label']}")



Eloszlás a 340. klaszterben:
- 36 db 12-os kép
- 62 db 50-os kép

Klaszter képei:
498: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04001.jpg | label=12
500: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04003.jpg | label=12
501: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04004.jpg | label=12
504: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04007.jpg | label=12
508: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04011.jpg | label=12
509: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04015.jpg | label=12
510: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04017.jpg | label=12
513: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04020.jpg | label=12
516: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04024.jpg | label=12
518: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04026.jpg | label=12
520: C:/Users/Adam/Desktop/applied_ml/dataset/train\12\image_04028.jpg | label=12
525: C:/Users/A

## Egy idegen elem hasonlósága a klaszter tagjaihoz

In [6]:
foreign_idx = 555
foreign_emb = selected_embeddings[foreign_idx].unsqueeze(0).numpy()

in_class_indices = [i for i in cluster if i != foreign_idx]
in_class_embs = torch.stack([selected_embeddings[i] for i in in_class_indices]).numpy()

similarities = cosine_similarity(foreign_emb, in_class_embs)[0]
print("Legkisebb / legnagyobb cosine similarity:", similarities.min(), "/", similarities.max())

Legkisebb / legnagyobb cosine similarity: 0.818612 / 0.9475775


## Szöveges caption embeddingek előkészítése

In [ ]:
caption_json_path = "../data/captions_till_50.json"
with open(caption_json_path, encoding="utf-8") as f:
    all_captions = json.load(f)

text_items = []
for class_id, image_dict in all_captions.items():
    if class_id not in included_classes:
        continue
    for img_name, caption in image_dict.items():
        text_items.append({
            "class_id": class_id,
            "image_name": img_name,
            "text": caption
        })

## Caption embeddingek

In [8]:
model = SiglipModel.from_pretrained("google/siglip-base-patch16-224").to(device)
processor = SiglipProcessor.from_pretrained("google/siglip-base-patch16-224")

captions = [item["text"] for item in text_items]
inputs = processor(text=captions, return_tensors="pt", padding=True, truncation=True).to(device)

with torch.no_grad():
    text_embeds = model.get_text_features(**inputs)
    text_embeds = text_embeds / text_embeds.norm(p=2, dim=-1, keepdim=True)
    text_embeds = text_embeds.cpu().numpy()

Using a slow image processor as `use_fast` is unset and a slow processor was saved with this model. `use_fast=True` will be the default behavior in v4.52, even if the model was saved with a slow processor. This will result in minor differences in outputs. You'll still be able to use a slow processor with `use_fast=False`.


## Gráfépítés és perkolációs klaszterezés szöveg alapján

In [9]:
epsilon = 0.15

similarity_matrix = cosine_similarity(text_embeds)
distance_matrix = 1 - similarity_matrix

G_txt = nx.Graph()
for i in range(len(text_items)):
    G_txt.add_node(i, class_id=text_items[i]["class_id"], image_name=text_items[i]["image_name"])
    for j in range(i + 1, len(text_items)):
        if distance_matrix[i][j] <= epsilon:
            G_txt.add_edge(i, j)

components_txt = list(nx.connected_components(G_txt))
print(f"Talált klaszterek száma (szöveg): {len(components_txt)}")
for idx, comp in enumerate(components_txt):
    comp_labels = {text_items[i]["class_id"] for i in comp}
    if len(comp_labels) > 1:
        print(f"⚠️ Klaszter {idx} több osztályt tartalmaz: {sorted(comp_labels)} (méret: {len(comp)})")
    else:
        print(f"Klaszter {idx}: tiszta ({list(comp_labels)[0]}) – {len(comp)} szöveg")

Talált klaszterek száma (szöveg): 602
⚠️ Klaszter 0 több osztályt tartalmaz: ['1', '2', '3'] (méret: 11)
Klaszter 1: tiszta (1) – 1 szöveg
Klaszter 2: tiszta (1) – 1 szöveg
Klaszter 3: tiszta (1) – 3 szöveg
Klaszter 4: tiszta (1) – 1 szöveg
Klaszter 5: tiszta (1) – 1 szöveg
Klaszter 6: tiszta (1) – 3 szöveg
Klaszter 7: tiszta (1) – 1 szöveg
Klaszter 8: tiszta (1) – 1 szöveg
Klaszter 9: tiszta (1) – 1 szöveg
Klaszter 10: tiszta (1) – 1 szöveg
Klaszter 11: tiszta (1) – 1 szöveg
Klaszter 12: tiszta (1) – 1 szöveg
Klaszter 13: tiszta (1) – 1 szöveg
⚠️ Klaszter 14 több osztályt tartalmaz: ['1', '4'] (méret: 2)
⚠️ Klaszter 15 több osztályt tartalmaz: ['10', '14', '29'] (méret: 125)
Klaszter 16: tiszta (10) – 1 szöveg
Klaszter 17: tiszta (10) – 1 szöveg
Klaszter 18: tiszta (10) – 1 szöveg
Klaszter 19: tiszta (10) – 1 szöveg
Klaszter 20: tiszta (10) – 1 szöveg
Klaszter 21: tiszta (10) – 1 szöveg
Klaszter 22: tiszta (11) – 58 szöveg
Klaszter 23: tiszta (11) – 2 szöveg
Klaszter 24: tiszta (11) –

In [12]:
from collections import Counter

target_idx = 0
cluster = components_txt[target_idx]

# Class ID-k összegyűjtése
cluster_class_ids = [text_items[i]["class_id"] for i in cluster]
label_counts = Counter(cluster_class_ids)

# Eloszlás
print(f"\nEloszlás a {target_idx}. klaszterben:")
for label, count in sorted(label_counts.items(), key=lambda x: int(x[0])):
    print(f"- {count} db {label}-os caption")

# ⚠️ Vegyes vagy tiszta klaszter?
if len(label_counts) > 1:
    print(f"\n⚠️ Figyelem! A klaszter több osztályt tartalmaz: {sorted(label_counts.keys())}")
else:
    print(f"\nTiszta klaszter: {list(label_counts.keys())[0]}")

# 📄 Részletes szövegek
print("\nKlaszter szövegek:")
for i in sorted(cluster):
    item = text_items[i]
    short_text = item["text"][:80].replace("\n", " ") + ("..." if len(item["text"]) > 80 else "")
    print(f"{i}: class_id={item['class_id']} | image={item['image_name']} | text=\"{short_text}\"")



Eloszlás a 0. klaszterben:
- 9 db 1-os caption
- 1 db 2-os caption
- 1 db 3-os caption

⚠️ Figyelem! A klaszter több osztályt tartalmaz: ['1', '2', '3']

Klaszter szövegek:
0: class_id=1 | image=image_06734.jpg | text="The image shows a close-up view of a pink flower in a natural setting. The flowe..."
6: class_id=1 | image=image_06741.jpg | text="This close-up shot features a pale pink flower with four petals. Delicate, darke..."
10: class_id=1 | image=image_06746.jpg | text="The image presents a close-up view of a single flower, prominently displayed aga..."
11: class_id=1 | image=image_06747.jpg | text="The image shows a close-up of a pink flower against a background of green foliag..."
12: class_id=1 | image=image_06748.jpg | text="The image shows a close-up view of a light pink flower with four petals. The pet..."
16: class_id=1 | image=image_06757.jpg | text="The image presents a close-up view of a single, fully open flower. The flower ha..."
22: class_id=1 | image=image_06768.j